# Partie 1 Fusion des données et création des indicateurs CROUS

## Objectif du notebook

Ce second notebook s’appuie sur les jeux de données **nettoyés et normalisés** du notebook précédent.  
L’objectif est désormais de **fusionner** les différentes sources d’information et de **créer des indicateurs synthétiques**
pour appuyer la prise de décision du **CROUS de l’Île-de-France**.

Nous travaillerons à partir de trois jeux de données principaux :

| Jeu de données | Contenu | Objectif |
|----------------|----------|-----------|
| **DVF** | Prix de vente immobilier (€/m²) | Identifier les zones où le foncier est abordable |
| **Loyers** | Loyer médian privé (€/m²/mois) | Repérer les communes où le marché privé est cher |
| **MESR (Île-de-France)** | Localisation et effectifs étudiants | Estimer la demande potentielle en logements étudiants |

Un quatrième jeu, issu de **l’INSEE (Population par âge)**, sera ajouté ultérieurement pour enrichir l’analyse démographique.


## Étapes de ce notebook

1. **Chargement des fichiers nettoyés** exportés depuis le notebook précédent  
2. **Vérification de la cohérence et des clés communes (`code_commune`)**  
3. **Fusion progressive** des trois sources (DVF, Loyers, MESR)  
4. **Création d’indicateurs utiles pour le CROUS**, tels que :
   - le ratio entre loyer et prix du foncier  
   - la densité d’établissements étudiants par commune  
   - la comparaison des coûts de marché par zone
5. **Export final** d’une base consolidée : `base_crous_idf.csv`



In [11]:
# Partie 1 Import des bibliothèques et chargement des fichiers nettoyés

import pandas as pd

# Lecture des trois fichiers exportés depuis le notebook précédent
dvf_commune = pd.read_csv("dvf_commune_clean.csv")
loyers_df = pd.read_csv("loyers_clean.csv")
mesr_idf = pd.read_csv("mesr_idf_clean.csv")

# Vérification rapide
print("=== Jeux de données nettoyés chargés avec succès ===")
print(f"DVF : {dvf_commune.shape[0]} lignes, {dvf_commune.shape[1]} colonnes")
print(f"Loyers : {loyers_df.shape[0]} lignes, {loyers_df.shape[1]} colonnes")
print(f"MESR (IDF) : {mesr_idf.shape[0]} lignes, {mesr_idf.shape[1]} colonnes")

# Aperçu rapide de chaque dataset
print("\nAperçu DVF :")
display(dvf_commune.head(3))

print("\nAperçu Loyers :")
display(loyers_df.head(3))

print("\nAperçu MESR :")
display(mesr_idf.head(3))


=== Jeux de données nettoyés chargés avec succès ===
DVF : 1283 lignes, 2 colonnes
Loyers : 34970 lignes, 14 colonnes
MESR (IDF) : 260 lignes, 29 colonnes

Aperçu DVF :


,code_commune,prix_median_m2
0,75101,13750.000000
1,75102,12262.726488
2,75103,12884.615385



Aperçu Loyers :


,id_zone,insee_c,libgeo,epci,dep,reg,loyer_m2_median,lwripm2,upripm2,typpred,nbobs_com,nbobs_mail,r2_adj,code_commune
0,1,36077,Fontguenand,200040558,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36077
1,1,36092,Langé,200040558,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36092
2,1,36123,Mézières-en-Brenne,243600343,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,8,521,"0,608561344447099",36123



Aperçu MESR :


,rentrée_universitaire,catégorie_détablissement,secteur_détablissement,code_uai_de_létablissement,sigle_de_létablissement,libellé_de_létablissement,libellé_complémentaire_de_létablissement,code_uai_de_la_composante,sigle_de_la_composante,libellé_de_la_composante,...,code_commune,commune,gps,degetu,degre_etudes,nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge,dont_femmes,dont_hommes,lat,lon
0,2023,"Écoles de commerce, gestion et vente",Privé,0590350K,EDHEC LILLE,EDHEC BUSINESS SCHOOL,MEMBRE FUPL,0755719J,EDHEC,CENTRE DE FORMATION CONTINUE,...,75102,Paris 2e,NaN,5.0,BAC + 5,387,162,225,NaN,NaN
1,2023,"Écoles de commerce, gestion et vente",Privé,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,MEMBRE FUPL,0922663V,IESEG BOULOGNE,INST ECO SC GEST BOULOGNE-BILL,...,92050,Nanterre,"48.89739112970496, 2.224693366221823",4.0,BAC + 4,1022,535,487,48.897391,2.224693
2,2023,"Écoles de commerce, gestion et vente",Privé,0751698N,PIGIER,ETAB ENSEIGNT SUPERIEUR PRIVE,LA COMPAGNIE DE FORMATION PIGI,0751698N,PIGIER,ETAB ENSEIGNT SUPERIEUR PRIVE,...,75119,Paris 19e,"48.896147655260954, 2.380225384944163",1.0,BAC + 1,212,133,79,48.896148,2.380225


# Partie 2  Fusion progressive des données

Nous allons maintenant combiner les trois sources principales autour du champ clé `code_commune`.

L’objectif est d’obtenir une table unique à l’échelle communale permettant de :
- comparer le **prix du foncier** et le **niveau des loyers privés**,  
- croiser ces informations avec la **présence d’établissements d’enseignement supérieur**,  
- préparer les futurs indicateurs de localisation stratégique pour le CROUS.

Nous commencerons par une fusion entre **DVF** et **Loyers**,  
puis nous y ajouterons les données du **MESR**.


## Étape 1  Fusion des données foncières et locatives

Dans cette première fusion, nous combinons :
- les **prix médians du foncier** issus du jeu **DVF**,  
- et les **loyers médians** du marché privé issus du jeu **SDES**.

L’objectif est d’obtenir un premier tableau unique contenant, pour chaque commune :
- le **prix moyen du m² à l’achat**,  
- le **loyer médian du m² à la location**,  
- et un **ratio loyer/prix foncier**, utile pour mesurer la tension entre achat et location.

Cette fusion se fera sur la clé commune `code_commune`.  
Seules les communes présentes dans les deux jeux seront conservées.


In [12]:
# Partie 2.1 – Fusion DVF + Loyers sur la clé code_commune

# Vérification du type de clé avant fusion
dvf_commune["code_commune"] = dvf_commune["code_commune"].astype(str)
loyers_df["code_commune"] = loyers_df["code_commune"].astype(str)

# Sélection des colonnes utiles pour éviter les doublons
loyers_light = loyers_df[["code_commune", "loyer_m2_median", "libgeo", "dep", "reg"]]

# Fusion interne (inner join) : conserve les communes présentes dans les deux datasets
dvf_loyers = pd.merge(dvf_commune, loyers_light, on="code_commune", how="inner")

# Création d’un indicateur économique : ratio loyer / prix foncier
dvf_loyers["ratio_loyer_prix"] = (
    dvf_loyers["loyer_m2_median"] / dvf_loyers["prix_median_m2"]
).round(4)

print(f"Fusion DVF + Loyers réussie : {dvf_loyers.shape[0]} communes communes.")
print("Aperçu :")
display(dvf_loyers.head(5))


Fusion DVF + Loyers réussie : 1283 communes communes.
Aperçu :


,code_commune,prix_median_m2,loyer_m2_median,libgeo,dep,reg,ratio_loyer_prix
0,75101,13750.000000,32.872899,Paris 1er Arrondissement,75,11,0.0024
1,75102,12262.726488,33.681936,Paris 2e Arrondissement,75,11,0.0027
2,75103,12884.615385,35.350405,Paris 3e Arrondissement,75,11,0.0027
3,75104,13660.000000,35.686505,Paris 4e Arrondissement,75,11,0.0026
4,75105,12882.307692,33.060947,Paris 5e Arrondissement,75,11,0.0026


### Interprétation de l’indicateur `ratio_loyer_prix`

Cet indicateur exprime le **niveau relatif du loyer par rapport au coût d’acquisition du foncier**.  
Il permet de comparer les communes entre elles selon leur **rentabilité locative potentielle**.

- Un ratio **élevé** → loyers chers par rapport aux prix d’achat → marché tendu (intéressant pour le CROUS).  
- Un ratio **faible** → loyers abordables ou foncier très cher → moindre intérêt économique.

Cet indicateur servira plus tard pour **hiérarchiser les zones** où une résidence universitaire publique
peut constituer une alternative viable au marché privé.


In [13]:
# Partie 2.2 – Contrôles statistiques sur la fusion DVF + Loyers

print("Résumé statistique du ratio loyer/prix foncier :")
display(dvf_loyers["ratio_loyer_prix"].describe(percentiles=[0.25, 0.5, 0.75]))

# Vérification du top et bottom 5
print("\nTop 5 des communes à ratio élevé (marché locatif tendu) :")
display(dvf_loyers.sort_values("ratio_loyer_prix", ascending=False).head(5)[
    ["code_commune", "libgeo", "prix_median_m2", "loyer_m2_median", "ratio_loyer_prix"]
])

print("\nBottom 5 des communes à ratio faible (foncier surévalué ou loyers bas) :")
display(dvf_loyers.sort_values("ratio_loyer_prix", ascending=True).head(5)[
    ["code_commune", "libgeo", "prix_median_m2", "loyer_m2_median", "ratio_loyer_prix"]
])


Résumé statistique du ratio loyer/prix foncier :


count    1283.000000
mean        0.005544
std         0.001175
min         0.000000
25%         0.005000
50%         0.005500
75%         0.006100
max         0.010900
Name: ratio_loyer_prix, dtype: float64


Top 5 des communes à ratio élevé (marché locatif tendu) :


,code_commune,libgeo,prix_median_m2,loyer_m2_median,ratio_loyer_prix
395,77396,Rupéreux,1250.000000,13.651531,0.0109
514,77523,Villuis,1368.421053,13.222727,0.0097
848,91215,Épinay-sous-Sénart,2083.333333,20.109818,0.0097
458,77461,Thénisy,1530.924370,14.518684,0.0095
268,77262,Louan-Villegruis-Fontaine,1418.271605,13.375857,0.0094



Bottom 5 des communes à ratio faible (foncier surévalué ou loyers bas) :


,code_commune,libgeo,prix_median_m2,loyer_m2_median,ratio_loyer_prix
1277,95675,Villeron,457053.347826,17.302896,0.0000
1029,93039,L'Île-Saint-Denis,430358.418605,20.829984,0.0000
1053,93079,Villetaneuse,353623.188406,20.533442,0.0001
1073,94043,Le Kremlin-Bicêtre,100000.000000,24.338524,0.0002
1016,93006,Bagnolet,35381.709677,22.449034,0.0006


## Étape 2  Fusion avec les établissements d’enseignement supérieur (

Nous intégrons maintenant les données du **MESR**, qui recensent les établissements d’enseignement supérieur
par commune, leur type et leurs effectifs étudiants.

L’objectif est d’ajouter une dimension « demande potentielle » à la base économique construite précédemment
(DVF + Loyers).

Nous allons :

1. Filtrer le jeu de données MESR pour ne conserver que les variables nécessaires.  
2. Calculer le **nombre d’établissements par commune** et le **total des étudiants inscrits**.  
3. Fusionner cette synthèse avec la table `dvf_loyers` via `code_commune`.


In [14]:
# Partie 2.3 – Agrégation du dataset MESR à l’échelle communale

# Sélection des colonnes utiles
mesr_light = mesr_idf[[
    "code_commune",
    "commune",
    "nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge"
]].copy()

# Nettoyage du type de code commune
mesr_light["code_commune"] = mesr_light["code_commune"].astype(str)

# Agrégation : nombre d’établissements et total étudiants par commune
mesr_agg = mesr_light.groupby("code_commune", as_index=False).agg(
    nb_etablissements=("commune", "count"),
    total_etudiants=("nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge", "sum")
)

# Vérification
print("MESR agrégé à l’échelle communale :")
display(mesr_agg.head(5))
print(f"Nombre de communes concernées : {mesr_agg.shape[0]}")


MESR agrégé à l’échelle communale :


,code_commune,nb_etablissements,total_etudiants
0,75056,1,49
1,75101,1,233
2,75102,1,387
3,75103,1,36
4,75104,1,139


Nombre de communes concernées : 260


## Fusion finale : DVF + Loyers + MESR

Nous fusionnons maintenant les trois jeux de données :
- `dvf_loyers` : base économique (prix foncier et loyers médians)  
- `mesr_agg` : présence étudiante par commune

La jointure se fait sur `code_commune` en utilisant un `left join`
pour conserver toutes les communes ayant des informations économiques, même sans présence d’établissements.


In [15]:
# Partie 2.4 – Fusion finale
base_crous = pd.merge(
    dvf_loyers,
    mesr_agg,
    on="code_commune",
    how="left"  # on garde toutes les communes DVF+Loyers
)

# Remplacement des valeurs manquantes (communes sans établissements)
base_crous["nb_etablissements"] = base_crous["nb_etablissements"].fillna(0).astype(int)
base_crous["total_etudiants"] = base_crous["total_etudiants"].fillna(0).astype(int)

# Vérification de la table finale
print(f"Base fusionnée : {base_crous.shape[0]} lignes, {base_crous.shape[1]} colonnes")
display(base_crous.head(5))


Base fusionnée : 1283 lignes, 9 colonnes


,code_commune,prix_median_m2,loyer_m2_median,libgeo,dep,reg,ratio_loyer_prix,nb_etablissements,total_etudiants
0,75101,13750.000000,32.872899,Paris 1er Arrondissement,75,11,0.0024,1,233
1,75102,12262.726488,33.681936,Paris 2e Arrondissement,75,11,0.0027,1,387
2,75103,12884.615385,35.350405,Paris 3e Arrondissement,75,11,0.0027,1,36
3,75104,13660.000000,35.686505,Paris 4e Arrondissement,75,11,0.0026,1,139
4,75105,12882.307692,33.060947,Paris 5e Arrondissement,75,11,0.0026,1,93


## Étape 3  Création d’indicateurs étudiants

À partir des variables agrégées du MESR, nous créons des indicateurs de **densité et attractivité étudiante** :

- `ratio_etudiants_par_etab` : nombre moyen d’étudiants par établissement dans la commune  
- `densite_etudiante_relative` : indicateur relatif comparant la taille de la population étudiante locale au nombre d’établissements  

Ces variables permettront d’identifier les zones où :
- la **demande potentielle** en logements étudiants est élevée,  
- mais où l’offre de logements CROUS est insuffisante.


In [16]:


# Moyenne d'étudiants par établissement (évite la division par 0)
base_crous["ratio_etudiants_par_etab"] = base_crous.apply(
    lambda x: x["total_etudiants"] / x["nb_etablissements"] if x["nb_etablissements"] > 0 else 0,
    axis=1
)

# Indicateur binaire : commune avec au moins un établissement
base_crous["presence_universitaire"] = (base_crous["nb_etablissements"] > 0).astype(int)

# Vérification
print("Aperçu des nouveaux indicateurs :")
display(base_crous[[
    "code_commune", "libgeo", "prix_median_m2", "loyer_m2_median",
    "nb_etablissements", "total_etudiants", "ratio_etudiants_par_etab", "presence_universitaire"
]].head(10))


Aperçu des nouveaux indicateurs :


,code_commune,libgeo,prix_median_m2,loyer_m2_median,nb_etablissements,total_etudiants,ratio_etudiants_par_etab,presence_universitaire
0,75101,Paris 1er Arrondissement,13750.000000,32.872899,1,233,233.0,1
1,75102,Paris 2e Arrondissement,12262.726488,33.681936,1,387,387.0,1
2,75103,Paris 3e Arrondissement,12884.615385,35.350405,1,36,36.0,1
3,75104,Paris 4e Arrondissement,13660.000000,35.686505,1,139,139.0,1
4,75105,Paris 5e Arrondissement,12882.307692,33.060947,1,93,93.0,1
5,75106,Paris 6e Arrondissement,15810.126582,36.705457,1,66,66.0,1
6,75107,Paris 7e Arrondissement,15106.382979,35.646884,1,55,55.0,1
7,75108,Paris 8e Arrondissement,13397.435897,34.901628,1,81,81.0,1
8,75109,Paris 9e Arrondissement,11810.810811,32.360469,1,354,354.0,1
9,75110,Paris 10e Arrondissement,10520.000000,30.070958,1,56,56.0,1


Nous disposons maintenant d’une base consolidée à l’échelle communale, combinant :
- le **coût du foncier** (DVF),
- le **niveau des loyers privés** (SDES),
- et la **présence étudiante** (MESR).

Cette base contient les indicateurs nécessaires pour :
- évaluer l’équilibre entre marché foncier et locatif,  
- repérer les communes attractives pour les étudiants,  
- identifier les zones où le développement de résidences CROUS est prioritaire.

L’étape suivante consistera à **enrichir cette base avec les données démographiques INSEE**
afin de mesurer la densité d’étudiants potentiels (18–25 ans) dans chaque commune.


# Partie 3  Intégration des données démographiques (INSEE)

Afin de compléter la vision économique et universitaire, nous ajoutons les données de l’INSEE
sur la **population par âge et par département**.

Ces informations permettront :
- d’évaluer le **poids des jeunes (18–25 ans)** dans chaque territoire,  
- et de construire un indicateur de **demande étudiante potentielle** à l’échelle communale.

Bien que le fichier INSEE soit fourni à l’échelle départementale, nous pourrons
l’associer aux communes via leur code de département (`dep`).


In [17]:


population_df = pd.read_excel(
    "population_par_sexe_age.xlsx",
    skiprows=3,  # saute les lignes d’introduction
    engine="openpyxl"
)

# Nettoyage des noms de colonnes
population_df.columns = [
    "dep", "nom_departement", "population_totale",
    "part_femmes_pct", "part_hommes_pct",
    "part_0_24_ans_pct", "part_25_59_ans_pct",
    "part_60_plus_pct", "part_75_plus_pct"
]

# Conversion du code département en chaîne de caractères pour la fusion
population_df["dep"] = population_df["dep"].astype(str).str.zfill(2)

print("Données INSEE chargées et nettoyées :")
display(population_df.head(5))


Données INSEE chargées et nettoyées :


,dep,nom_departement,population_totale,part_femmes_pct,part_hommes_pct,part_0_24_ans_pct,part_25_59_ans_pct,part_60_plus_pct,part_75_plus_pct
0,01,Ain,688626.0,50.7,49.3,29.7,44.8,25.5,9.3
1,02,Aisne,518817.0,51.1,48.9,28.9,41.7,29.4,10.6
2,03,Allier,332599.0,51.7,48.3,24.4,39.0,36.6,14.9
3,04,Alpes-de-Haute-Provence,169806.0,51.5,48.5,24.1,39.9,35.9,14.3
4,05,Hautes-Alpes,142006.0,51.1,48.9,23.1,41.1,35.8,14.0


## Étape 3.2  Fusion avec la base CROUS

Nous fusionnons les données démographiques à la base `base_crous`
via le code département (`dep`), déjà présent dans les données de loyers.

Cela permet d’associer à chaque commune :
- la **population totale** du département,
- la **part des jeunes de 0 à 24 ans**, utilisée comme approximation
  du poids de la population étudiante potentielle.


In [18]:
# Fusion sur le code département
base_crous = pd.merge(
    base_crous,
    population_df[["dep", "population_totale", "part_0_24_ans_pct"]],
    on="dep",
    how="left"
)

# Calcul d’un indicateur de densité étudiante potentielle
base_crous["potentiel_etudiant_pct"] = (
    base_crous["part_0_24_ans_pct"] * base_crous["presence_universitaire"]
)

# Vérification du résultat
print("Base enrichie avec les données INSEE :")
print(base_crous.shape)
display(base_crous.head(5))


Base enrichie avec les données INSEE :
(1283, 14)


,code_commune,prix_median_m2,loyer_m2_median,libgeo,dep,reg,ratio_loyer_prix,nb_etablissements,total_etudiants,ratio_etudiants_par_etab,presence_universitaire,population_totale,part_0_24_ans_pct,potentiel_etudiant_pct
0,75101,13750.000000,32.872899,Paris 1er Arrondissement,75,11,0.0024,1,233,233.0,1,2048472.0,25.6,25.6
1,75102,12262.726488,33.681936,Paris 2e Arrondissement,75,11,0.0027,1,387,387.0,1,2048472.0,25.6,25.6
2,75103,12884.615385,35.350405,Paris 3e Arrondissement,75,11,0.0027,1,36,36.0,1,2048472.0,25.6,25.6
3,75104,13660.000000,35.686505,Paris 4e Arrondissement,75,11,0.0026,1,139,139.0,1,2048472.0,25.6,25.6
4,75105,12882.307692,33.060947,Paris 5e Arrondissement,75,11,0.0026,1,93,93.0,1,2048472.0,25.6,25.6


## Étape 3.3  Indicateurs synthétiques pour le CROUS

À partir des variables combinées, nous pouvons construire plusieurs indicateurs utiles
pour la priorisation des investissements publics :

- **`ratio_loyer_prix`** : tension économique entre marché locatif et foncier  
- **`presence_universitaire`** : existence d’au moins un établissement  
- **`ratio_etudiants_par_etab`** : capacité moyenne d’accueil par établissement  
- **`potentiel_etudiant_pct`** : proportion estimée de jeunes dans le département  

Nous ajoutons également un indicateur composite pour évaluer la **pertinence d’une implantation CROUS** :
une commune est jugée prioritaire si :
1. Les loyers sont élevés,
2. Le foncier est relativement abordable,
3. Il existe une présence universitaire significative.


In [19]:
import numpy as np


# --- Vérifs colonnes minimales ---
for col, default in [
    ("ratio_loyer_prix", np.nan),
    ("prix_median_m2", np.nan),
    ("loyer_m2_median", np.nan),
    ("nb_etablissements", 0),
    ("total_etudiants", 0),
    ("potentiel_etudiant_pct", np.nan),
]:
    if col not in base_crous.columns:
        base_crous[col] = default

# --- Helpers de normalisation ---
def qrank(series):
    """Rang percentile (0..1), ignore NaN."""
    return series.rank(pct=True, method="average")

def minmax(series):
    s = series.astype(float)
    mn, mx = s.min(), s.max()
    if pd.isna(mn) or pd.isna(mx) or mx == mn:
        return pd.Series(0.0, index=s.index)
    return (s - mn) / (mx - mn)

# 1) Scores élémentaires (0..1)
# Tension locative : ratio élevé = mieux
s_ratio   = qrank(base_crous["ratio_loyer_prix"].fillna(base_crous["ratio_loyer_prix"].median()))

# Foncier : moins cher = mieux  -> 1 - rang(prix)
s_foncier = 1 - qrank(base_crous["prix_median_m2"].fillna(base_crous["prix_median_m2"].median()))

# Masse étudiante : log pour réduire l’effet des extrêmes, puis min-max
etudiants_log = np.log1p(base_crous["total_etudiants"].fillna(0))
s_etudiants   = minmax(etudiants_log)

# Démographie (faiblement pondérée car départementale)
if base_crous["potentiel_etudiant_pct"].notna().any():
    s_demo = qrank(base_crous["potentiel_etudiant_pct"].fillna(base_crous["potentiel_etudiant_pct"].median()))
else:
    s_demo = pd.Series(0.0, index=base_crous.index)

# Bonus binaire si présence universitaire (au moins 1 établissement)
presence_uni = (base_crous["nb_etablissements"].fillna(0) > 0).astype(int)

# 2) Pondérations pragmatiques (somme = 1)
#   - Ratio (tension locative) : 0.40
#   - Foncier (coût d’investissement) : 0.30
#   - Étudiants (demande effective) : 0.25
#   - Démographie (potentiel futur, dataset départemental) : 0.05
w_ratio, w_foncier, w_etud, w_demo = 0.40, 0.30, 0.25, 0.05

score_continu = (
    w_ratio   * s_ratio +
    w_foncier * s_foncier +
    w_etud    * s_etudiants +
    w_demo    * s_demo
)

# Petit coup de pouce si présence universitaire (bonus additif léger)
# 0.03 ≈ 3 points sur une échelle 0..100, sans faire exploser le ranking
score_continu = score_continu + 0.03 * presence_uni

# 3) Échelle lisible : 0..100
base_crous["score_priorite_crous_continu"] = (score_continu * 100).round(1)

# 4) Discrétisation 0..3 pour lecture managériale (quartiles)
q25, q50, q75 = score_continu.quantile([0.25, 0.50, 0.75])

def to_band(x):
    if x <= q25: return 0
    if x <= q50: return 1
    if x <= q75: return 2
    return 3

base_crous["score_priorite_crous"] = score_continu.apply(to_band).astype(int)

# 5) Contrôles et aperçu
print("Distribution score discret (0..3) :")
display(base_crous["score_priorite_crous"].value_counts().sort_index())

print("\nRésumé score continu :")
display(base_crous["score_priorite_crous_continu"].describe())

print("\nTop 10 communes (score continu) :")
cols_show = [
    "code_commune", "libgeo",
    "prix_median_m2", "loyer_m2_median", "ratio_loyer_prix",
    "nb_etablissements", "total_etudiants", "potentiel_etudiant_pct",
    "score_priorite_crous_continu", "score_priorite_crous"
]
display(base_crous.sort_values("score_priorite_crous_continu", ascending=False).head(10)[cols_show])


Distribution score discret (0..3) :


score_priorite_crous
0    321
1    321
2    320
3    321
Name: count, dtype: int64


Résumé score continu :


count    1283.000000
mean       40.526033
std        18.446911
min         2.000000
25%        26.000000
50%        39.900000
75%        55.600000
max        83.900000
Name: score_priorite_crous_continu, dtype: float64


Top 10 communes (score continu) :


,code_commune,libgeo,prix_median_m2,loyer_m2_median,ratio_loyer_prix,nb_etablissements,total_etudiants,potentiel_etudiant_pct,score_priorite_crous_continu,score_priorite_crous
1021,93014,Clichy-sous-Bois,2292.462908,19.113300,0.0083,1,53,34.3,83.9,3
456,77459,Sourdun,1892.857143,13.375857,0.0071,1,33,32.4,83.0,3
977,91692,Les Ulis,2443.650794,19.352375,0.0079,1,55,32.6,82.3,3
697,78440,Les Mureaux,1920.331737,17.855139,0.0093,1,10,31.7,82.1,3
291,77285,Le Mée-sur-Seine,2142.857143,17.452290,0.0081,1,21,32.4,81.6,3
853,91228,Évry-Courcouronnes,2405.063291,18.669464,0.0078,1,32,32.6,80.9,3
407,77409,Saint-Germain-Laval,2130.666667,14.482791,0.0068,1,31,32.4,79.5,3
139,77126,Congis-sur-Thérouanne,2572.388060,17.485657,0.0068,1,57,32.4,77.9,3
600,78208,Élancourt,3037.931034,20.301305,0.0067,1,456,31.7,77.8,3
310,77305,Montereau-Fault-Yonne,2430.555556,16.733082,0.0069,1,25,32.4,77.3,3


## Construction du score de priorité CROUS

### Objectif

Dans cette section, nous nous intéressons à l'attribut `score_priorite_crous`, l’indicateur central du projet qui permet de **hiérarchiser les communes d’Île-de-France** selon leur **pertinence pour l’implantation de logements étudiants**.

Elle introduit une **approche continue et pondérée**, permettant une évaluation plus fine des territoires.

### Méthodologie

Le score repose sur quatre dimensions principales, issues des bases de données nettoyées précédemment :

| Domaine | Indicateur clé | Source | Interprétation |
|----------|----------------|---------|----------------|
| **Immobilier** | `prix_m2_median` | DVF – DGFiP | Coût du foncier. Un prix bas favorise la construction de logements. |
| **Marché locatif** | `ratio_loyer_prix` | Calcul interne (loyer / prix du m²) | Mesure la tension locative : plus le ratio est élevé, plus les loyers sont disproportionnés par rapport au foncier, donc plus la demande étudiante est forte. |
| **Université** | `total_etudiants`, `nb_etablissements` | MESR – Effectifs d’étudiants | Reflète la demande réelle et la densité universitaire dans la commune. |
| **Démographie** | `potentiel_etudiant_pct` | INSEE – Population par âge | Estime la part de jeunes (0–24 ans) dans la population locale, donc le potentiel de demande future. |

### Étapes du calcul

1. **Normalisation des variables (0 à 1)**  
   Pour rendre comparables des indicateurs exprimés dans des unités différentes (€/m², nombre, pourcentage),  
   on convertit chaque variable en **score relatif** sur une échelle commune de 0 à 1 à l’aide de rangs ou de transformations logarithmiques :
   - un prix foncier bas donne une valeur proche de 1 ;
   - un ratio loyer/prix élevé donne une valeur proche de 1 ;
   - un grand nombre d’étudiants donne une valeur proche de 1 ;
   - un pourcentage élevé de jeunes donne une valeur proche de 1.

2. **Pondération selon la stratégie du CROUS**  
   Chaque dimension n’a pas la même importance dans la décision finale.  
   Nous appliquons les pondérations suivantes, qui reflètent la logique d’investissement public :

   | Dimension | Poids | Justification |
   |------------|--------|---------------|
   | Tension locative (`ratio_loyer_prix`) | **0.40** | Indique la rareté du logement étudiant sur le marché privé |
   | Foncier abordable (`prix_m2_median`) | **0.30** | Impacte directement le coût d’investissement |
   | Présence universitaire (`total_etudiants`) | **0.25** | Reflète la demande réelle actuelle |
   | Démographie (`potentiel_etudiant_pct`) | **0.05** | Facteur de long terme, utilisé avec prudence car dataset départemental |

3. **Ajout d’un bonus binaire : présence universitaire**  
   Les communes qui possèdent **au moins un établissement d’enseignement supérieur** obtiennent un **léger bonus de 3 %** sur leur score total car elles présentent une probabilité plus forte d’occupation des logements.

4. **Calcul du score continu (0 à 100)**  
   Les quatre dimensions pondérées sont additionnées pour former un score composite, ensuite ramené sur une échelle de 0 à 100.  
   Ce score peut être interprété comme un **indice de priorité** relatif entre toutes les communes d’Île-de-France.

5. **Transformation en classes de priorité (0 à 3)**  
   Pour faciliter la lecture et la communication auprès des décideurs publics,  
   le score continu est discrétisé en quatre catégories selon ses quartiles :

   | Score | Priorité | Interprétation |
   |:--:|:--|:--|
   | **0** | Très faible | Commune peu adaptée ou déjà saturée |
   | **1** | Faible | Opportunité limitée |
   | **2** | Moyenne | Potentiel modéré |
   | **3** | **Forte priorité** | Foncier abordable, loyers élevés et présence universitaire marquée |


### Lecture et interprétation des résultats

Une fois le calcul effectué, deux colonnes sont créées dans le jeu de données :
- `score_priorite_crous_continu` : le score numérique entre 0 et 100, utile pour des comparaisons fines entre communes ;
- `score_priorite_crous` : la version simplifiée (0 à 3) pour la priorisation managériale.

**Plus le score est élevé, plus la commune présente un intérêt stratégique pour le CROUS.**


Ce score représente une **synthèse cohérente et quantitative** des principaux facteurs influençant la faisabilité et la pertinence des projets de résidences universitaires :

- Il équilibre **coût, tension locative, demande actuelle et future**.
- Il est **reproductible** sur d’autres régions grâce à sa structure modulaire.
- Il constitue une **base solide** pour la cartographie et la hiérarchisation territoriale des priorités CROUS.


# Partie 4 Export de la base consolidée CROUS Île-de-France

Nous disposons désormais d’une base complète intégrant :
- les **prix du foncier (DVF)**,
- les **loyers du marché privé (SDES)**,
- la **présence et les effectifs d’étudiants (MESR)**,
- et les **données démographiques (INSEE)**.

Cette table servira de base unique pour les visualisations et les analyses spatiales
réalisées dans le prochain notebook.


In [20]:
# Export du jeu de données consolidé
base_crous.to_csv("base_crous_idf.csv", index=False, encoding="utf-8-sig")

print("=== Export réussi ===")
print("Fichier généré : base_crous_idf.csv")
print("Base complète pour la visualisation et l’analyse du CROUS.")


=== Export réussi ===
Fichier généré : base_crous_idf.csv
Base complète pour la visualisation et l’analyse du CROUS.


# Clôture du Notebook 2 – Indicateurs CROUS

Le travail de fusion et d’enrichissement est maintenant terminé.  
Nous disposons d’une base commune qui combine à la fois :
- des **indicateurs économiques** (foncier, loyers),
- des **indicateurs académiques** (présence étudiante),
- et des **indicateurs démographiques** (part des jeunes).

Le **Notebook 3** sera dédié à :
- la **visualisation** de ces indicateurs sous forme de cartes et graphiques,  
- et à la **priorisation visuelle** des communes à fort potentiel pour de nouvelles résidences universitaires.
